# Deploy operators

In [ ]:
oc apply -k operators/
echo DONE

In [ ]:
oc -n openshift-operators wait --timeout=300s subscription/cert-manager --for=condition=CatalogSourcesUnhealthy=False
oc -n openshift-operators wait --timeout=300s subscription/cert-manager --for=condition=BundleUnpacking=False
oc -n openshift-operators wait --timeout=300s csv \
  -l operators.coreos.com/cert-manager.openshift-operators="" --for=jsonpath={.status.phase}='Succeeded'
echo DONE

In [ ]:
oc apply -k kustomize/bundles/cert-manager/app/
echo DONE

In [ ]:
oc apply -k kustomize/bundles/postgresql/base/
oc -n openshift-operators --timeout=300s wait subscription/postgresql \
   --for=condition=CatalogSourcesUnhealthy=False
oc -n openshift-operators --timeout=300s wait csv \
  -l operators.coreos.com/postgresql.openshift-operators="" --for=jsonpath={.status.phase}='Succeeded'
echo DONE

In [ ]:
oc apply -k kustomize/bundles/postgresql/app/
oc -n postgres wait --timeout=300s postgrescluster/postgres --for=condition=PgBackRestRepoHostReady=True
echo DONE

In [ ]:
oc apply -k kustomize/bundles/keycloak/base/
oc -n keycloak --timeout=300s wait subscription/keycloak-operator \
   --for=condition=CatalogSourcesUnhealthy=False
oc -n keycloak --timeout=300s wait csv \
  -l operators.coreos.com/keycloak-operator.keycloak="" --for=jsonpath={.status.phase}='Succeeded'

In [ ]:
oc -n postgres get secret postgres-pguser-keycloak -o json \
    | jq 'del(.metadata["namespace","creationTimestamp","resourceVersion","selfLink","uid","ownerReferences"])' \
    | oc -n keycloak apply -f -


In [ ]:
oc apply -k kustomize/bundles/keycloak/app/
oc -n keycloak wait --timeout=300s keycloak/keycloak --for=condition=Ready=True